In [1]:
from create_tables import main as main_create_tables
from etl import main as main_etl

In [ ]:
# Uncomment to set up the tables and load the data 
# main_create_tables()
# main_etl()

In [6]:
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries
import boto3
import json
import pandas as pd

# Set parser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# Get data
access_key_id          = config.get("AWS","AWS_ACCESS_KEY_ID")
secret_access_key      = config.get("AWS","AWS_SECRET_ACCESS_KEY")

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

BUCKET = config.get("S3", "BUCKET")

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, "-", "-", "-", "-", DWH_PORT, "-"]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,-
4,DWH_DB,-
5,DWH_DB_USER,-
6,DWH_DB_PASSWORD,-
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,-


### Create the clients

In [7]:
# Create the clients
ec2 = boto3.resource('ec2', 
                      region_name="us-west-2",
                      aws_access_key_id=access_key_id,
                      aws_secret_access_key=secret_access_key)

iam = boto3.client('iam',
                    region_name="us-west-2",
                    aws_access_key_id=access_key_id,
                    aws_secret_access_key=secret_access_key)

redshift = boto3.client('redshift',
                        region_name="us-west-2",
                        aws_access_key_id=access_key_id,
                        aws_secret_access_key=secret_access_key,
                       )

s3 = boto3.resource('s3',
                    region_name="us-west-2",
                    aws_access_key_id=access_key_id,
                    aws_secret_access_key=secret_access_key,
                   )

### Load the files in the Bucket

In [8]:
# Look at files
udacity_dend =  s3.Bucket('udacity-dend')

In [5]:
# Log Data
log_data_files = udacity_dend.objects.filter(Prefix="log-data")
log_data_files = [f.key for f in log_data_files][1:]
print(f"#Log data files:{len(log_data_files)}")

#Log data files:30


In [ ]:
# song Data
song_data_files = udacity_dend.objects.filter(Prefix="song-data")
song_data_files = [f.key for f in song_data_files][1:]
print(f"#song data files:{len(song_data_files)}")

### Create IAM Role

In [9]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

# print(roleArn)

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name redshiftRole already exists.
1.2 Attaching Policy
1.3 Get the IAM role ARN


### Create RedShift Cluster

In [5]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        # NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists


In [11]:
cluster_properties = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
cols = ["ClusterIdentifier", "NodeType", "ClusterStatus", 
        "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']

pd.DataFrame([(c,cluster_properties[c]) 
              if c not in ['MasterUsername', 'DBName', 'Endpoint', 'VpcId']
              else (c,'-') for c in cols],
             columns=['Property', 'Value'])

,Property,Value
0,ClusterIdentifier,redshift-cluster-1
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,-
4,DBName,-
5,Endpoint,-
6,NumberOfNodes,1
7,VpcId,-


In [12]:
DWH_ENDPOINT = cluster_properties['Endpoint']['Address']
DWH_ROLE_ARN = cluster_properties['IamRoles'][0]['IamRoleArn']

# print(f"DWH_ENDPOINT :: {DWH_ENDPOINT}")
# print(f"DWH_ROLE_ARN :: {DWH_ROLE_ARN}")

print("DWH_ENDPOINT :: -")
print("DWH_ROLE_ARN :: -")

DWH_ENDPOINT :: -
DWH_ROLE_ARN :: -


In [8]:
try:
    vpc = ec2.Vpc(id=cluster_properties['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-0e477f12fc330fc4e')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


In [13]:
# Delete the redshift cluster
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'redshift-cluster-1',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'awsconsole',
  'DBName': 'postgressql',
  'Endpoint': {'Address': 'redshift-cluster-1.cgr8egvzdlvo.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2022, 5, 16, 17, 7, 37, 223000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0e477f12fc330fc4e',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-05cc39a4e7a316983',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'sat:09:30-sat:10:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,